In [ ]:
import os
import sys
import random
import itertools
import inspect
import multiprocess
import pandas as pd
import numpy as np
from tqdm import tqdm
from math import sqrt
from  warnings import simplefilter
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.exceptions import ConvergenceWarning

In [ ]:
random.seed(0)
simplefilter('ignore', category = ConvergenceWarning)

In [ ]:
df_training = pd.read_csv('../../datasets/titanic_training_processed.csv')
df_test = pd.read_csv('../../datasets/titanic_test_processed.csv')

In [ ]:
df_training.head()

In [ ]:
df_test.head()

In [ ]:
columns = df_training.columns[2:]
X_train = df_training[columns].values
X_test = df_test[columns].values
y_train = df_training['Survived'].values

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_train.shape

## No feature selection

We have to still apply 10-fold cross validation to select the kernel and the value of C. For the polynomial and RBF kernels we also need to determine the value of their respective parameters  

In [ ]:
# generating sets for 10-fold cross validation
indexes = list(range(len(df_training)))
random.shuffle(indexes)
folds = []
for i in range(10):
    folds.append([])
for i in range(len(indexes)):
    folds[i % 10].append(indexes[i])

In [ ]:
def produce_training_test_set(df_training, train_indexes, test_indexes, column_indexes):
    columns = df_training.columns[column_indexes]
    datasets = {}
    datasets['X_train'] = df_training.iloc[train_indexes][columns].values
    datasets['X_test'] = df_training.iloc[test_indexes][columns].values
    datasets['y_train'] = df_training.iloc[train_indexes]['Survived'].values
    datasets['y_test'] = df_training.iloc[test_indexes]['Survived'].values
    
    return datasets

In [ ]:
def evaluate(datasets, C, kernel, kernel_params):
    kwargs = {}
    if kernel == 'poly':
        kwargs['degree'] = kernel_params['degree']
        kwargs['coef0'] = kernel_params['coef0']
       
    clf = SVC(max_iter = 50000, C = C, gamma = 'auto', kernel = kernel, **kwargs)
    clf.fit(datasets['X_train'], datasets['y_train'])
    y_pred = clf.predict(datasets['X_test'])
    return sqrt(np.sum(np.power(np.array(y_pred) - np.array(datasets['y_test']), 2)))

In [ ]:
def k_fold_cross_validation(df_training, folds, column_indexes, C, kernel, kernel_params):
    error = 0
    
    for k in range(10):
        train_indexes = []
        for j in range(10):
            if j == k:
                test_indexes = folds[j]
            else:
                train_indexes = train_indexes + folds[j]
                
        datasets = produce_training_test_set(df_training, train_indexes, test_indexes, column_indexes)
        
        error = error + evaluate(datasets, C, kernel, kernel_params)
        
    return error / 10.0

In [ ]:
# Results were very similar for gamma = scale and gamma = auto
# No good results with degree = 1
# Manually setting the ranges and steps after testing multiple times so we do not get so many combinations
# for the feature selection case
C = np.arange(0.2, 5.2, 0.2).tolist()
kernel = ['linear', 'poly', 'rbf']
degree = [2, 3]
coef0 = np.arange(0, 3.2, 0.2).tolist()

poly_params = list(itertools.product(*[degree, coef0]))

comb = list(itertools.product(*[C, ['linear'], [None], [None]]))
comb.extend(list(itertools.product(*[C, ['rbf'], [None], [None]])))
comb.extend(list(itertools.product(*[C, ['poly'], degree, coef0])))

column_indexes = list(range(2, 62)) # All columns
minimum = sys.float_info.max

errors = pd.DataFrame(data = comb, columns = ['C', 'kernel', 'degree', 'coef0'])
errors['error'] = np.nan

for i in tqdm(range(len(errors))):
    errors.loc[i, 'error'] = k_fold_cross_validation(df_training,
                                                     folds,
                                                     column_indexes,
                                                     errors['C'].loc[i],
                                                     errors['kernel'].loc[i],
                                                     {'degree': errors['degree'].loc[i],
                                                      'coef0': errors['coef0'].loc[i]})

In [ ]:
errors = errors.sort_values(by = 'error')
errors.head(5)

In [ ]:
fig, ax = plt.subplots()
errors_linear = errors[errors.kernel == 'linear'].sort_values(by = 'C')
ax.plot(errors_linear.C, errors_linear.error)
ax.set_xlabel('C')
ax.set_ylabel('RMSE')
ax.set_title('Linear model')
ax.grid(True)

In [ ]:
fig, ax = plt.subplots()
errors_rbf = errors[errors.kernel == 'rbf'].sort_values(by = 'C')
ax.plot(errors_rbf.C, errors_rbf.error)
ax.set_xlabel('C')
ax.set_ylabel('RMSE')
ax.set_title('RBF kernel')
ax.grid(True)

In [ ]:
fig, ax = plt.subplots(1, len(degree))
errors_poly = errors[errors.kernel == 'poly']
for d in degree:
    i = degree.index(d)
    errors_d = errors_poly[errors_poly.degree == d].pivot(index='C', 
                                                          columns='coef0', 
                                                          values='error')
    im = ax[i].imshow(errors_d, cmap = 'viridis', extent=[errors.C.min(), 
                                                          errors.C.max(), 
                                                          errors.coef0.min(), 
                                                          errors.coef0.max()])
    fig.colorbar(im, ax = ax[i])
    ax[i].set_xlabel('C')
    ax[i].set_ylabel('coef0')
    ax[i].set_title('poly kernel - degree = ' + str(d))
fig.set_figwidth(12)
fig.set_figheight(6)

In [ ]:
best_C = errors.C.values[0]
best_kernel = errors.kernel.values[0]
best_degree = errors.degree.values[0]
best_coef0 = errors.coef0.values[0]
clf = SVC(C = best_C, 
          gamma = 'auto', 
          kernel = best_kernel, 
          degree = best_degree,
          coef0 = best_coef0)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
y_test = clf.predict(X_test)

In [ ]:
submission = df_test.copy()
submission['Survived'] = y_test
submission = submission[['PassengerId', 'Survived']]

In [ ]:
submission.head()

In [ ]:
if not os.path.exists('./submissions/'):
    os.makedirs('./submissions/')

In [ ]:
submission.to_csv('./submissions/05_svm.csv', index = False)

My submission to Kaggle produced a 78.95% test prediction accuracy. This is almost as high as what I my best result *with feature selection* to this date.

## Feature selection - forward selection

Unfortunately I cannot apply full model selection during feature selection. I will use the best model parameter values obtained by applying model selection to the full mmodel. 

In [ ]:
def k_fold_cross_validation_unpack(args):
    from tmp_func import k_fold_cross_validation
    
    return k_fold_cross_validation(args[0],
                                   args[1],
                                   args[2],
                                   args[3][0],
                                   args[3][1],
                                   {'degree': args[3][2], 'coef0': args[3][3]})

In [ ]:
with open('tmp_func.py', 'w') as f:
    f.write('from math import sqrt\n')
    f.write('from sklearn.svm import SVC\n')
    f.write('import numpy as np\n')
    f.write(inspect.getsource(evaluate))
    f.write(inspect.getsource(produce_training_test_set))
    f.write(inspect.getsource(k_fold_cross_validation))

In [ ]:
# Forward selection
pending = list(range(2, 62))
model = []
min_error = sys.float_info.max
num_processes = multiprocess.cpu_count() - 1

while len(pending) > 0:
    prev_error = min_error
    
    for i in tqdm(pending):
        new_model = model + [i]
        parameters = itertools.product([df_training], [folds], [new_model], comb)
        with multiprocess.Pool(processes = num_processes) as pool:
            errors = pool.map(k_fold_cross_validation_unpack, parameters)
       
        minimum = min(errors) 
        if minimum < min_error:
            min_error = minimum
            best_model = new_model
            feature = i
            best_comb = comb[np.argmin(errors)]
        
    if min_error < prev_error:
        print('Selecting feature ' + 
              df_training.columns[feature] + 
              '(C = ' + 
              str(best_comb[0]) +
              ', kernel = ' +
              best_comb[1] +
              ', degree = ' + 
              str(best_comb[2]) +
              ', coef0 = ' +
              str(best_comb[3]) + 
              ') - error decreased to ' +
              str(min_error))
        model = model + [feature]
        pending.remove(feature)
    else:
        print('END')
        break

In [ ]:
model_forward = model
columns = df_training.columns[model_forward]
X_train = df_training[columns].values
X_test = df_test[columns].values
y_train = df_training['Survived'].values

In [ ]:
kwargs = dict()
if best_comb[1] == 'poly':
    kwargs['degree'] = best_comb[2]['degree']
    kwargs['coef0'] = best_comb[3]['coef0']
       
clf = SVC(C = best_comb[0], gamma = 'auto', kernel = best_comb[1], **kwargs)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
y_test = clf.predict(X_test)

In [ ]:
submission = df_test.copy()
submission['Survived'] = y_test
submission = submission[['PassengerId', 'Survived']]

In [ ]:
submission.head()

In [ ]:
submission.to_csv('./submissions/05_svm_forward_selection.csv', index = False)

Even though we got a lower error with this subset of features as compared as to the whole set (RMSE = 3.77 vs RMSE = 3.9), this submissions only got a 76.55% accuracy on Kaggle vs. the 78.95% I got with the complete set. 

## Feature selection - backward elimination

In [ ]:
# backward elimination
model = list(range(2, 62))
# Using the results for the full model's best model
min_error = k_fold_cross_validation(df_training, 
                                    folds, 
                                    column_indexes, 
                                    best_C, 
                                    best_kernel, 
                                    {'degree': best_degree, 
                                     'coef0': best_coef0})
num_processes = multiprocess.cpu_count() - 1

while len(model) > 0:
    prev_error = min_error
    
    for i in tqdm(model):
        new_model = model[:]
        new_model.remove(i)
        parameters = itertools.product([df_training], [folds], [new_model], comb)
        
        with multiprocess.Pool(processes = num_processes) as pool:
            errors = pool.map(k_fold_cross_validation_unpack, parameters)
        
        minimum = min(errors)
        
        if minimum < min_error:
            min_error = minimum
            best_model = new_model
            feature = i
            best_comb = comb[np.argmin(errors)]
            
    if min_error < prev_error:
        print('Removing feature ' + 
              df_training.columns[feature] + 
              '(C = ' + 
              str(best_comb[0]) +
              ', kernel = ' +
              best_comb[1] +
              ', degree = ' + 
              str(best_comb[2]) +
              ', coef0 = ' +
              str(best_comb[3]) + 
              ') - error decreased to ' +
              str(min_error))
        model = best_model
    else:
        print('END')
        break

In [ ]:
model_backward = model
columns = df_training.columns[model_backward]
X_train = df_training[columns].values
X_test = df_test[columns].values
y_train = df_training['Survived'].values

In [ ]:
kwargs = dict()
if best_comb[1] == 'poly':
    kwargs['degree'] = best_comb[2]
    kwargs['coef0'] = best_comb[3]
       
clf = SVC(C = best_comb[0], gamma = 'auto', kernel = best_comb[1], **kwargs)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
y_test = clf.predict(X_test)

In [ ]:
submission = df_test.copy()
submission['Survived'] = y_test
submission = submission[['PassengerId', 'Survived']]

In [ ]:
submission.head()

In [ ]:
submission.to_csv('./submissions/05_svm_backward_elimination.csv', index = False)

This submission produced the same accuracy as the complete model. 

In [ ]:
os.remove('./tmp_func.py')